#### Imports

In [9]:
import os
import re
import skrf as rf
import pandas as pd

#### Paths

In [10]:
script_directory = os.getcwd()#obtem diretório atual
s2ps_path = os.path.join(script_directory,'..','..','oil_data','oil_data') #cria string caminho para todos os .s2p

#### Regex

In [11]:
regex = r'SAW_06_02_2024_OIL_5W40_(\d+)\.S2P'

In [12]:
def read_s2p_data(regex, path):
    dfs = []#lista para ocncatenar todos os dicts de cada medição 
    for filename in os.listdir(path):
        match = re.match(regex, filename)
        if match:
            #obtem informação do nome do arquivo
            oil_name = "5W40"
            measurement_number = int(match.group(1))
            
            #carrega arquivo
            path_to_oil = os.path.join(path, filename)
            s2p_file = rf.Network(path_to_oil)
            
            #extrai frequência e parâmetros S do objeto Network
            frequency = s2p_file.f
            s_parameters = s2p_file.s_db[:,1,0]
            phase = s2p_file.s_deg[:,1,0]
            
            #Dataframe com informações selecionadas
            individual_oil_dict = {
                'OIL NAME': oil_name,
                'OIL TYPE': 'SYNTHETIC',
                'MEASUREMENT NUMBER': measurement_number,
                'FREQUENCY (MHZ)':frequency/1000000,
                'S21 (db)' : s_parameters,
                'PHASE (DEG)': phase,
            }
            df = pd.DataFrame.from_dict(individual_oil_dict)
            df = df.nlargest(3, 'S21 (db)')
            dfs.append(df)
            
    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

#### Read Dataframes

In [13]:
df_synthetic = read_s2p_data(regex, s2ps_path)

In [14]:
#assegura somente óleo de 5W40 lido
for name in df_synthetic["OIL NAME"]:
    if name != "5W40":
        print("Diferente de 5W40")

In [15]:
#cria .csv caso precise no futuro.
path_save_csv = os.path.join(script_directory,'..','..', 'oil_data','oil_data','CSVs')
if not os.path.exists(path_save_csv):
   os.makedirs(path_save_csv)
df_synthetic.to_csv(path_save_csv + '/5W40_measurements.csv')

In [16]:
df_synthetic.info

<bound method DataFrame.info of   OIL NAME   OIL TYPE  MEASUREMENT NUMBER  FREQUENCY (MHZ)  S21 (db)  \
0     5W40  SYNTHETIC                   1       118.876888 -57.34062   
1     5W40  SYNTHETIC                   1       118.898890 -57.44957   
2     5W40  SYNTHETIC                   1       118.888889 -57.47217   
3     5W40  SYNTHETIC                   2       118.876888 -57.19274   
4     5W40  SYNTHETIC                   2       118.878888 -57.42343   
5     5W40  SYNTHETIC                   2       118.868887 -57.49381   
6     5W40  SYNTHETIC                   3       118.946895 -57.34936   
7     5W40  SYNTHETIC                   3       118.870887 -57.56983   
8     5W40  SYNTHETIC                   3       118.880888 -57.61698   

   PHASE (DEG)  
0     179.4519  
1     161.2556  
2     167.0403  
3     176.9566  
4     174.6133  
5    -176.6842  
6     140.2582  
7    -170.9507  
8    -174.3236  >